In [2]:
import os, glob, pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, Callback

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from scripts import common_quick_plots as qp

np.random.seed(9)

### Phase 1 - Problem Definition  

#### 1.1 Broad Goals  

I have multiple small animals underfoot while I'm coding, cats and dogs.  While my dog is canine shaped and behaved, she is a small breed and ruff-ly the same size or smaller than my cat.  

#### 1.2 Data Source  

This project combines two open source datasets to create a single sample:  
    1. the [Stanford Dogs Dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) which includes a limited number of samples for each category and over 120 separate dog breeds.  
    2. the [Kaggle Cat Breeds Dataset](https://www.kaggle.com/ma7555/cat-breeds-dataset) which includes a limited number of samples for 67 different cat breeds

#### 1.3 Problem Statement 

While identifying which pet is at my feet is an easy task for me as a human, it becomes a larger challenge for a computer. This project aims to build a binary image classification model to check whether a pet is a cat or a dog from a photo, with a stretch goal of applying real-time testing to a webcam.

### Phase 2 - Data Gathering  
####    2.1 Loading Files

In [15]:
imgs = []

def load_resize(file, size = (160, 160)):
    # load file
    try:
        img_arr = cv2.imread(file)
        #convert to color array
        img_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
        #reshape to uniform size 160x160x3
        return cv2.resize(img_rgb, size)
    except:
        print(f"Invalid Path")

for path in glob.glob('./data/*'):
    species = os.path.basename(path)
    for breed in glob.glob(path +'/*'):
        # pull a simplified name
        if species == 'cat':
            breed_name = breed.split('/')[-1]
            for file in glob.glob(breed + '/*.jpg')[:350]: #limit to 350 per cat breed
                file_name = os.path.basename(file)
                # load file
                image = load_resize(file)
                imgs.append({'species' : species, 
                             'breed': breed_name, 
                             'image' : image
                            })
        else:
            breed_name = breed.split('-')[-1]
            for file in glob.glob(breed + '/*.jpg')[:150]: #limit to 150 per dog breed
                file_name = os.path.basename(file)
                # load file
                image = load_resize(file)
                imgs.append({'species' : species, 
                             'breed': breed_name, 
                             'image' : image
                            })

                
#convert list to dataframe
df = pd.DataFrame.from_dict(imgs)

Image not found
Image not found


In [16]:
df.head()

,species,breed,image
0,cat,Chausie,"[[[205, 206, 210], [206, 207, 211], [212, 213,..."
1,cat,Chausie,"[[[46, 62, 75], [45, 61, 76], [45, 58, 74], [4..."
2,cat,Chausie,"[[[163, 195, 236], [147, 179, 220], [129, 162,..."
3,cat,Chausie,"[[[248, 206, 210], [252, 211, 215], [247, 206,..."
4,cat,Chausie,"[[[181, 169, 153], [185, 173, 157], [188, 176,..."


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33104 entries, 0 to 33103
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   species  33104 non-null  object
 1   breed    33104 non-null  object
 2   image    33102 non-null  object
dtypes: object(3)
memory usage: 776.0+ KB


### Phase 3 - Exploratory Data Analysis  

#### 3.1 Dataset Shape

In [18]:
df['species'].value_counts()

dog    17997
cat    15107
Name: species, dtype: int64

In [19]:
df['species'].value_counts(normalize = True)

dog    0.54365
cat    0.45635
Name: species, dtype: float64

#### 3.2 Sample Images

#### 3.3 Principal Component Analysis

### Phase 4 - Modeling  

#### 4.1 Train/Test/Split

In [67]:
X = []
y = []
for path in glob.glob('./data/*'):
    species = os.path.basename(path)
    for breed in glob.glob(path +'/*'):
        # pull a simplified name
        if species == 'cat':
            breed_name = breed.split('/')[-1]
            for file in glob.glob(breed + '/*.jpg')[:350]: #limit to 350 per cat breed
                file_name = os.path.basename(file)
                # load file
                try:
                    img_arr = cv2.imread(file)
                    #convert to color array
                    img_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
                    #reshape to uniform size 160x160x3
                    new_arr =  cv2.resize(img_rgb, (160, 160))
                    X.append(new_arr)
                    y.append(1)
                except:
                    print(f"Invalid Path")
        else:
            breed_name = breed.split('-')[-1]
            for file in glob.glob(breed + '/*.jpg')[:150]: #limit to 150 per dog breed
                file_name = os.path.basename(file)
                # load file
                image = load_resize(file)
                try:
                    img_arr = cv2.imread(file)
                    #convert to color array
                    img_rgb = cv2.cvtColor(img_arr, cv2.COLOR_BGR2RGB)
                    #reshape to uniform size 160x160x3
                    new_arr =  cv2.resize(img_rgb, (160, 160))
                    X.append(new_arr)
                    y.append(0)
                except:
                    print(f"Invalid Path")

Invalid Path
Invalid Path


In [76]:
X = np.array(X)
y = np.array(y)
X.shape, y.shape

((33102, 160, 160, 3), (33102,))

In [78]:
X = X.astype('float32')
y = utils.to_categorical(y, 2)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 9)

#### 4.2 Convolutional Neural Net

In [89]:
cnn = Sequential()
# Starting Layer
cnn.add(Conv2D(filters = 512,
               kernel_size = (3, 3),
               activation = 'relu',
               input_shape = (160, 160, 3)))
cnn.add(MaxPooling2D(pool_size = (2,2)))
#second convolutional layer
cnn.add(Conv2D(filters=256,            
                     kernel_size=(3, 3),        
                     activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

#third convolutional layer
cnn.add(Conv2D(filters=128,            
                     kernel_size=(3, 3),        
                     activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

#flatten the metrics to fit into the Dense layers 
cnn.add(Flatten())
cnn.add(Dense(32, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(512, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(256, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(128, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(64, activation = 'relu'))
cnn.add(Dropout(0.3))
cnn.add(Dense(32, activation = 'relu'))
cnn.add(Dense(1, activation = 'softmax'))
cnn.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
early_stop = EarlyStopping(patience = 5, restore_best_weights = True)

In [90]:
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 158, 158, 512)     14336     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 79, 79, 512)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 77, 77, 256)       1179904   
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 38, 38, 256)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 36, 36, 128)       295040    
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 18, 18, 128)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 41472)            

In [91]:
res = cnn.fit(X_train, y_train,
             batch_size = 64,
             validation_data = (X_test, y_test),
             epochs = 100,
             callbacks = [early_stop],
             verbose = 1)

Epoch 1/100
  5/388 [..............................] - ETA: 1:29:56 - loss: 7.6246 - accuracy: 0.5000

KeyboardInterrupt: 